<a href="https://colab.research.google.com/github/Inventrohyder/death-app/blob/main/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>